In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../dataset/biendata/data_with_province_update_20200229.csv',index_col=['date','cases'])

In [3]:
df

湖北    广东    河南    浙江    湖南   安徽   江西   江苏   重庆  \
date      cases                                                                 
2020/1/20 total_confirmed     270    14     0     0     0    0    0    0    0   
          total_recoveries     25     0     0     0     0    0    0    0    0   
          total_deaths          6     0     0     0     0    0    0    0    0   
2020/1/21 total_confirmed     375    26     1     1     1    3    2    0    5   
          total_recoveries     28     0     0     0     0    0    0    0    0   
...                           ...   ...   ...   ...   ...  ...  ...  ...  ...   
2020/2/28 total_recoveries  28895   973  1162   987   841  868  811  519  422   
          total_deaths       2727     7    21     1     4    6    1    0    6   
2020/2/29 total_confirmed   66907  1349  1272  1205  1018  990  935  631  576   
          total_recoveries  31187  1009  1185  1027   853  868  831  529  438   
          total_deaths       2761     7    22     1     4    6    1    0    6   

                             山东  ...  甘肃  吉林  内蒙古  新疆  宁夏  香港  台湾  青海  澳门  西藏  
date      cases                  ...                                           
2020/1/20 total_confirmed     0  ...   0   0    0   0   0   0   0   0   0   0  
          total_recoveries    0  ...   0   0    0   0   0   0   0   0   0   0  
          total_deaths        0  ...   0   0    0   0   0   0   0   0   0   0  
2020/1/21 total_confirmed     1  ...   0   0    0   0   0   0   0   0   0   0  
          total_recoveries    0  ...   0   0    0   0   0   0   0   0   0   0  
...                         ...  ...  ..  ..  ...  ..  ..  ..  ..  ..  ..  ..  
2020/2/28 total_recoveries  411  ...  82  73   46  52  68  30   9  18   8   1  
          total_deaths        6  ...   2   1    0   3   0   2   1   0   0   0  
2020/2/29 total_confirmed   756  ...  91  93   75  76  73  95  39  18  10   1  
          total_recoveries  431  ...  82  75   51  62  69  33   9  18   8   1  
          total_deaths        6  ...   2   1    0   3   0   2   1   0   0   0  

[123 rows x 34 columns]

In [4]:
temp_df = df.stack().reset_index()

In [5]:
temp_df.columns=['date','cases','province','count']

In [6]:
temp_df = temp_df.set_index(['date','province'])

In [7]:
temp_df.to_csv('../dataset/processed/province_temp.csv',encoding='gbk')

In [8]:
# pd.pivot(xx, index='cases', columns=['date','province'], values='count')

In [9]:
temp_df = temp_df.reset_index()

In [10]:
all_df = temp_df.groupby(['date','cases'],as_index=False)['count'].sum()

In [11]:
all_df['province']='全国'

In [12]:
temp_df = pd.concat([temp_df,all_df],sort=False)

In [13]:
temp_df['data&province']=temp_df.apply(lambda row:row['date']+'#'+row['province'],axis=1)

In [14]:
stack_df=temp_df[['data&province','cases','count']]

In [15]:
stack_df.set_index(['data&province','cases'],inplace=True) # 为 unstack 做准备

In [16]:
unstack_df = stack_df.unstack().reset_index()

In [17]:
target = unstack_df.T.reset_index().drop(['level_0'],axis=1).T

In [18]:
target.columns = target.iloc[0]

In [19]:
target.drop(target.index[0],inplace=True)

In [20]:
target.columns = ['dt&loc', 'total_confirmed', 'total_deaths', 'total_recoveries']

In [21]:
target['total_confirmed'] = target['total_confirmed'].apply(lambda x:int(x))
target['total_deaths'] = target['total_deaths'].apply(lambda x:int(x))
target['total_recoveries'] = target['total_recoveries'].apply(lambda x:int(x))

In [22]:
# 计算康复率
target['recovery_rate'] = target.apply(lambda row:-1 if int(row['total_confirmed'])==0 else 
                                       100*float(row['total_recoveries'])/float(row['total_confirmed']),axis=1)

In [23]:
# 恢复省份和日期
target['date'] = target['dt&loc'].apply(lambda x:x.split("#")[0])
target['province'] = target['dt&loc'].apply(lambda x:x.split("#")[1])

In [24]:
target['date'] = pd.to_datetime(target['date'])

In [25]:
target['type']='治愈率'

In [26]:
target

,dt&loc,total_confirmed,total_deaths,total_recoveries,recovery_rate,date,province,type
0,2020/1/20#上海,2,0,0,0.000000,2020-01-20,上海,治愈率
1,2020/1/20#云南,0,0,0,-1.000000,2020-01-20,云南,治愈率
2,2020/1/20#全国,291,6,25,8.591065,2020-01-20,全国,治愈率
3,2020/1/20#内蒙古,0,0,0,-1.000000,2020-01-20,内蒙古,治愈率
4,2020/1/20#北京,5,0,0,0.000000,2020-01-20,北京,治愈率
...,...,...,...,...,...,...,...,...
1430,2020/2/9#重庆,473,2,66,13.953488,2020-02-09,重庆,治愈率
1431,2020/2/9#陕西,213,0,27,12.676056,2020-02-09,陕西,治愈率
1432,2020/2/9#青海,10,0,3,30.000000,2020-02-09,青海,治愈率
1433,2020/2/9#香港,38,1,0,0.000000,2020-02-09,香港,治愈率


In [27]:
output = target[['province','type','recovery_rate','date']]

In [28]:
output.columns=['name','type','value','date']

In [29]:
# drop 确诊人数为0的行
output = output[output['value']!=-1]

In [30]:
output.to_csv('../dataset/processed/provice_recovery_rate.csv',index=False)

In [31]:
output

,name,type,value,date
0,上海,治愈率,0.000000,2020-01-20
2,全国,治愈率,8.591065,2020-01-20
4,北京,治愈率,0.000000,2020-01-20
13,广东,治愈率,0.000000,2020-01-20
22,湖北,治愈率,9.259259,2020-01-20
...,...,...,...,...
1430,重庆,治愈率,13.953488,2020-02-09
1431,陕西,治愈率,12.676056,2020-02-09
1432,青海,治愈率,30.000000,2020-02-09
1433,香港,治愈率,0.000000,2020-02-09
